In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

mnist = mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Normalize the pixels in 0.0~1.0 float
train_images, test_images = train_images / 255.0, test_images / 255.0

print(train_images.shape)
print(train_labels.shape)

In [ ]:
# Define a model
class mnist_lenet(tf.keras.Model):
    def __init__(self):
        super(mnist_lenet, self).__init__()
        self.conv1 = layers.Conv2D(filters=10, kernel_size=[3,3], input_shape = (28,28,1), activation= 'relu')
        self.pool1 = layers.MaxPooling2D(2, 2)
        self.conv2 = layers.Conv2D(filters=20, kernel_size=[3,3], activation= 'relu')
        self.pool2 = layers.MaxPooling2D(2, 2)
        self.conv3 = layers.Conv2D(filters=30, kernel_size=[3,3], activation= 'relu')
        self.flat = layers.Flatten()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(10, activation='softmax')


    def call(self, x):
        net = self.conv1(x)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.dense2(net)
        return net

In [ ]:
# Train & save model in frozen(.pb) format.

my_model = mnist_lenet()
my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
my_model.fit(train_images, train_labels, batch_size=1, epochs=5, verbose=1)
my_model.summary()
my_model.save('mnist_lenet')

In [ ]:
# Load model and convert it for TensorFlow Lite (.tflite format)

import tensorflow as tf
import numpy as np

model_path = '/Users/{USER}/mnist_lenet/'
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()


In [ ]:
import pathlib

# Create folder to save model.
tflite_models_dir = pathlib.Path("/tmp/mnist_lenet/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mnist_lenet.tflite"
tflite_model_file.write_bytes(tflite_model)

In [ ]:
# For Edge TPU.
# You must quntize your model in INT8 precision.

# Get representative data set for post-quantization.
# The representative data set prevents accuracy drop while quantization.
def representative_data_gen():
    for image in train_images[:1000]:  # Use a subset of the dataset
        # Resize the image to the input shape of your model
        image = tf.image.resize(image, (28, 28))
        image = np.expand_dims(image, axis=0)
        yield [image]

model_path = '/Users/{USER}/mnist_lenet/'
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_model = converter.convert()

In [ ]:
import pathlib

# Create folder to save model.
tflite_models_dir = pathlib.Path("/tmp/mnist_lenet/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"mnist_lenet_quant.tflite"
tflite_model_file.write_bytes(tflite_model)

# Now, you can convert your quzntized model for Edge TPU with edgetpu_compiler.
# follow https://coral.ai/docs/edgetpu/compiler/#download